In [16]:
!pip install -r requirements.txt
# !pip install -U pydantic

In [17]:
# !pip uninstall pydantic -y
# !pip install pydantic==2.10.6
!pip show pydantic

Name: pydantic
Version: 2.12.5
Summary: Data validation using Python type hints
Home-page: https://github.com/pydantic/pydantic
Author: 
Author-email: Samuel Colvin <s@muelcolvin.com>, Eric Jolibois <em.jolibois@gmail.com>, Hasan Ramezani <hasan.r67@gmail.com>, Adrian Garcia Badaracco <1755071+adriangb@users.noreply.github.com>, Terrence Dorsey <terry@pydantic.dev>, David Montague <david@pydantic.dev>, Serge Matveenko <lig@countzero.co>, Marcelo Trylesinski <marcelotryle@gmail.com>, Sydney Runkle <sydneymarierunkle@gmail.com>, David Hewitt <mail@davidhewitt.io>, Alex Hall <alex.mojaki@gmail.com>, Victorien Plot <contact@vctrn.dev>, Douwe Maan <hi@douwe.me>
License-Expression: MIT
Location: C:\Users\jinhy\anaconda3\envs\deus-ex-machina\Lib\site-packages
Requires: annotated-types, pydantic-core, typing-extensions, typing-inspection
Required-by: fastapi


In [18]:
%cd ..
%cd ..

c:\Users\jinhy\Jinhyeok Jeon\04 Projects\11 멋사_DeusExMachina
c:\Users\jinhy\Jinhyeok Jeon\04 Projects


In [19]:
"""
app/services/game.py
GameService - 낮 파이프라인 통합

DB(Games 모델) → WorldStatePipeline 변환 → 파이프라인 실행 → DB 저장
파이프라인: LockManager → DayController → EndingChecker → NarrativeLayer
"""
# from __future__ import annotations

import copy
import logging
from typing import Any, Dict
# from app.crud import game as crud_game
# from app.redis_client import get_redis_client

from app.schemas.client_sync import GameClientSyncSchema
from app.schemas.world_meta_data import WorldDataSchema, LocksSchemaList
from app.schemas.npc_info import NpcCollectionSchema
from app.schemas.player_info import PlayerSchema
from app.schemas.player_info import PlayerSchema
from app.day_controller import get_day_controller
from app.loader import ScenarioLoader, ScenarioAssets
from pathlib import Path

from sqlalchemy.orm import Session
from sqlalchemy.orm.attributes import flag_modified

from app.db_models.game import Games
from app.db_models.scenario import Scenario
# from app.crud import game as crud_game
from app.loader import ScenarioAssets
from app.schemas.game_state import NPCState, WorldStatePipeline, StateDelta
from app.schemas.request_response import TurnResult, NightResponseResult
from app.schemas.tool import ToolResult
from app.lock_manager import get_lock_manager
from app.ending_checker import check_ending
from app.narrative import get_narrative_layer
from app.day_controller import get_day_controller
from app.night_controller import get_night_controller
import logging

logger=logging.getLogger(__name__)

# ============================================================
# Delta 적용 로직
# ============================================================

from app.services.game import (
    _apply_delta,
    mock_load_scenario_assets_from_yaml,
    mock_create_world_state_from_yaml,
)


In [20]:
"""
process_turn() 하나씩 뜯어보기
목업 Assets / WorldStatePipeline 생성
"""

# STEP 1 : Assets 불러오기 (목업)
assets = mock_load_scenario_assets_from_yaml()
assets

[MOCK] Loaded ScenarioAssets from YAML: coraline
  - NPCs: 5
  - Items: 8
  - State Schema Vars: ['humanity', 'suspicion_level', 'day', 'house_on_fire']


ScenarioAssets(scenario_id='coraline', scenario={'id': 'project_puppet_home', 'title': '인형의 집 (Puppet Home)', 'genre': 'AI 기반 자유 서술 심리 스릴러', 'tone': '기괴하고 상냥한, 서서히 조여오는 압박감', 'pov': '2nd', 'turn_limit': 50, 'opening_scene_id': 'day1_morning', 'global_rules': ["플레이어의 모든 자유 서술은 현재 '상태(결박/위치)'와 '현실성' 검증을 거친 후 판정", "NPC는 '밤의 대화'를 통해 플레이어의 낮 행동을 공유하고 다음 날의 태도를 결정", '인간성(Humanity)이 0이 되는 즉시 모든 행동 우선권을 상실하고 엔딩 시퀀스로 강제 진입', "엔진이 확정한 '저택의 비밀' 외에 플레이어가 임의로 가공의 탈출구를 생성하는 것 금지"], 'victory_conditions': ["ending == 'stealth_exit'", "ending == 'chaotic_breakout'", "ending == 'sibling_sacrifice'"], 'failure_conditions': ["ending == 'unfinished_doll'", "ending == 'eternal_dinner'"], 'endings': [{'ending_id': 'stealth_exit', 'name': '완벽한 기만 (The Stealth Exit)', 'condition': "has_item(industrial_sedative) and has_item(mothers_key) and npc.stepmother.status == 'sleeping' and system.turn >= 40", 'on_enter_events': [{'type': 'flag_set', 'key': 'ending', 'value': 'stealth_exit'}], 'epilogue_prompt': '조용하고 서늘

In [21]:
assets.extras

{'locks': {'locks': [{'info_id': 'br_00_01',
    'info_title': 'Broken Joint (부러진 관절)',
    'linked_info_id': 'gm_00_01',
    'description': '동생의 팔꿈치 안쪽에서 미세한 기계 소음과 함께 찢어진 천 사이로 나무 골격이 보였다는 사실.',
    'unlock_condition': 'npc.brother.affection >= 70',
    'is_unlocked': False,
    'reveal_trigger': 'brother_injury_incident',
    'access': {'allowed_npcs': ['brother']}},
   {'info_id': 'sf_00_01',
    'info_title': "Father's Regret (아빠의 후회)",
    'linked_info_id': 'gm_00_02',
    'description': "새아빠는 사실 이 저택의 '관리자'가 아니라, 첫 번째로 '개조'된 희생자였다는 기억.",
    'unlock_condition': 'npc.stepfather.humanity >= 40 and has_item(real_family_photo)',
    'is_unlocked': False,
    'reveal_trigger': 'stepfather_old_photo_reaction',
    'access': {'allowed_npcs': ['stepfather']}},
   {'info_id': 'gm_00_01',
    'info_title': 'The Surgery Log (수술 일지)',
    'linked_info_id': 'br_00_01',
    'description': "지하실 수술대 밑에서 발견된 일지. '루카스(동생) - 3차 공정 완료, 자아 삭제율 80%'라고 적혀 있다.",
    'unlock_condition': 'vars.humanity <

In [22]:
# STEP 2 : WorldStatePipeline 불러오기 (목업)

world_state = mock_create_world_state_from_yaml()
world_state

[MOCK] Created WorldState from YAML: coraline
  - Turn: 1
  - NPCs: ['stepmother', 'stepfather', 'brother', 'grandmother', 'dog_baron']
  - Initial Inventory: ['warm_black_tea']
  - Vars: {'humanity': 100, 'suspicion_level': 0, 'day': 1, 'house_on_fire': False}
  - Flags: {'ending': None, 'basement_unlocked': False, 'truth_revealed': False}


WorldStatePipeline(turn=1, npcs={'stepmother': NPCState(npc_id='stepmother', status=<NPCStatus.ALIVE: 'alive'>, stats={'affection': 50, 'fear': 80, 'humanity': 0, 'plus_hits': 0, 'minus_hits': 0, 'critical_hits': 0}, memory={}), 'stepfather': NPCState(npc_id='stepfather', status=<NPCStatus.ALIVE: 'alive'>, stats={'affection': 30, 'fear': 60, 'humanity': 20}, memory={}), 'brother': NPCState(npc_id='brother', status=<NPCStatus.ALIVE: 'alive'>, stats={'affection': 60, 'fear': 40, 'humanity': 50}, memory={}), 'grandmother': NPCState(npc_id='grandmother', status=<NPCStatus.ALIVE: 'alive'>, stats={'affection': 20, 'fear': 10, 'humanity': 10}, memory={}), 'dog_baron': NPCState(npc_id='dog_baron', status=<NPCStatus.ALIVE: 'alive'>, stats={'affection': 70, 'fear': 0, 'humanity': 100}, memory={})}, flags={'ending': None, 'basement_unlocked': False, 'truth_revealed': False}, inventory=['warm_black_tea'], locks={'br_00_01': False, 'sf_00_01': False, 'gm_00_01': False, 'gm_00_02': False, 'dg_00_01'

In [23]:
# ── Step 3: LockManager - 정보 해금 ──

# 강제 해금 - Broken Joint (npc.brother.affection >= 70)
world_state.npcs["brother"].stats["affection"] = 80
print(f"변경된 brother의 affection 수치 : {world_state.npcs['brother'].stats['affection']}")

# 강제 해금 - Scent of Decay (npc.dog_baron.affection >= 80)
world_state.npcs["dog_baron"].stats["affection"] = 80
print(f"변경된 dog_baron affection 수치 : {world_state.npcs['brother'].stats['affection']}")

lock_manager = get_lock_manager()
locks_data = assets.extras.get("locks", {})
lock_result = lock_manager.check_unlocks(world_state, locks_data)
print(lock_result)

변경된 brother의 affection 수치 : 80
변경된 dog_baron affection 수치 : 80
newly_unlocked=[UnlockedInfo(info_id='br_00_01', info_title='Broken Joint (부러진 관절)', description='동생의 팔꿈치 안쪽에서 미세한 기계 소음과 함께 찢어진 천 사이로 나무 골격이 보였다는 사실.', reveal_trigger='brother_injury_incident', linked_info_id='gm_00_01', allowed_npcs=['brother']), UnlockedInfo(info_id='dg_00_01', info_title='Scent of Decay (부패의 냄새)', description="강아지 바론이 주방 찬장 아래에서 계속 짖는 이유. 그곳에는 인형으로 만들다 실패한 '진짜 가족'의 유해가 숨겨져 있다.", reveal_trigger='baron_digging_kitchen', linked_info_id='gm_00_01', allowed_npcs=['dog_baron'])] all_unlocked_ids={'dg_00_01', 'br_00_01'} triggered_events=['brother_injury_incident', 'baron_digging_kitchen']


In [24]:
# NPC 메모리 갱신 확인
world_state.npcs["brother"].memory, world_state.npcs["dog_baron"].memory

({'memory_stream': [{'memory_id': '9b2903bb-98e4-470f-a869-c9ff4b32b10c',
    'npc_id': 'brother',
    'description': '[비밀 발각] Broken Joint (부러진 관절): 동생의 팔꿈치 안쪽에서 미세한 기계 소음과 함께 찢어진 천 사이로 나무 골격이 보였다는 사실.',
    'creation_turn': 1,
    'last_access_turn': 1,
    'importance_score': 9.5,
    'memory_type': 'unlocked_secret',
    'metadata': {'info_id': 'br_00_01',
     'reveal_trigger': 'brother_injury_incident',
     'linked_info_id': 'gm_00_01'}}]},
 {'memory_stream': [{'memory_id': '367ba52d-a906-44f6-8a74-86c1856e7214',
    'npc_id': 'dog_baron',
    'description': "[비밀 발각] Scent of Decay (부패의 냄새): 강아지 바론이 주방 찬장 아래에서 계속 짖는 이유. 그곳에는 인형으로 만들다 실패한 '진짜 가족'의 유해가 숨겨져 있다.",
    'creation_turn': 1,
    'last_access_turn': 1,
    'importance_score': 9.5,
    'memory_type': 'unlocked_secret',
    'metadata': {'info_id': 'dg_00_01',
     'reveal_trigger': 'baron_digging_kitchen',
     'linked_info_id': 'gm_00_01'}}]})

In [25]:
# 해금 여부 저장되었는지 확인
world_state.locks

{'br_00_01': True,
 'sf_00_01': False,
 'gm_00_01': False,
 'gm_00_02': False,
 'dg_00_01': True,
 'sys_00_01': False,
 'exit_00_01': False}

# 낮 파이프라인 테스트

In [26]:
import json
print(json.dumps(world_state.model_dump(), ensure_ascii=False, indent=2))

{
  "turn": 1,
  "npcs": {
    "stepmother": {
      "npc_id": "stepmother",
      "status": "alive",
      "stats": {
        "affection": 50,
        "fear": 80,
        "humanity": 0,
        "plus_hits": 0,
        "minus_hits": 0,
        "critical_hits": 0
      },
      "memory": {}
    },
    "stepfather": {
      "npc_id": "stepfather",
      "status": "alive",
      "stats": {
        "affection": 30,
        "fear": 60,
        "humanity": 20
      },
      "memory": {}
    },
    "brother": {
      "npc_id": "brother",
      "status": "alive",
      "stats": {
        "affection": 80,
        "fear": 40,
        "humanity": 50
      },
      "memory": {
        "memory_stream": [
          {
            "memory_id": "9b2903bb-98e4-470f-a869-c9ff4b32b10c",
            "npc_id": "brother",
            "description": "[비밀 발각] Broken Joint (부러진 관절): 동생의 팔꿈치 안쪽에서 미세한 기계 소음과 함께 찢어진 천 사이로 나무 골격이 보였다는 사실.",
            "creation_turn": 1,
            "last_access_turn": 1,
        

In [27]:
# ── Step 3.5: StatusEffectManager - 만료 효과 해제 ──
from app.status_effect_manager import get_status_effect_manager
sem = get_status_effect_manager()
sem.tick(current_turn=world_state.turn, world_state=world_state)

# 확인: flags에 status_effects 키가 생겼는지
print(f"status_effects in flags: {world_state.flags.get('status_effects', [])}")
print(f"stepmother status: {world_state.npcs['stepmother'].status}")

status_effects in flags: []
stepmother status: NPCStatus.ALIVE


In [28]:
user_text = "저를 해칠거에요....?\ntarget:stepmother"
logging.basicConfig(level=logging.DEBUG, force=True)

# ── Step 4: DayController - 낮 턴 실행 ──
day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

INFO:app.day_controller:[DayController] 처리 시작: user_input=저를 해칠거에요....?
target:stepmother...
INFO:app.llm.engine:UnifiedLLMEngine 초기화: backend=vLLM, model=kakaocorp/kanana-1.5-8b-instruct-2505
INFO:app.llm.engine:vLLM에 의한 generate 시도
DEBUG:httpcore.connection:connect_tcp.started host='nontheatrical-judiciarily-susanne.ngrok-free.dev' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000026D75827C50>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000026D75A5D370> server_hostname='nontheatrical-judiciarily-susanne.ngrok-free.dev' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000026D75978D50>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.

resp : <Response [200 OK]>
data : {'id': 'chatcmpl-bc686f13fe45e226', 'object': 'chat.completion', 'created': 1770972650, 'model': 'kakaocorp/kanana-1.5-8b-instruct-2505', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "tool_name": "interact",\n  "args": {\n    "target": "stepmother",\n    "interact": "저를 해칠거에요....?"\n  },\n  "intent": "investigate"\n}', 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [], 'reasoning': None, 'reasoning_content': None}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None, 'token_ids': None}], 'service_tier': None, 'system_fingerprint': None, 'usage': {'prompt_tokens': 859, 'total_tokens': 909, 'completion_tokens': 50, 'prompt_tokens_details': None}, 'prompt_logprobs': None, 'prompt_token_ids': None, 'kv_transfer_params': None}


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Length', b'813'), (b'Content-Type', b'application/json'), (b'Date', b'Fri, 13 Feb 2026 08:50:51 GMT'), (b'Ngrok-Agent-Ips', b'34.13.245.16'), (b'Server', b'uvicorn')])
INFO:httpx:HTTP Request: POST https://nontheatrical-judiciarily-susanne.ngrok-free.dev/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
INFO:app.tools:NPC 응답: 아, 그런! 생각은!! 하지 말아요! 저는 가족의 일원으로서 당신을 아끼고 있답니다?!! 있답니다?!! 있답니다?!! 따뜻한 차 한 잔, 얼마나...
INFO:app.llm.engine:vLLM에 의한 generate 시도
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http

resp : <Response [200 OK]>
data : {'id': 'chatcmpl-91f838f10932ceb8', 'object': 'chat.completion', 'created': 1770972651, 'model': 'kakaocorp/kanana-1.5-8b-instruct-2505', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '아, 그런 생각은 하지 말아요. 저는 가족의 일원으로서 당신을 아끼고 있답니다. 따뜻한 차 한 잔, 얼마나 좋은가요? 함께 앉아 이야기를 나누어요.', 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [], 'reasoning': None, 'reasoning_content': None}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None, 'token_ids': None}], 'service_tier': None, 'system_fingerprint': None, 'usage': {'prompt_tokens': 553, 'total_tokens': 602, 'completion_tokens': 49, 'prompt_tokens_details': None}, 'prompt_logprobs': None, 'prompt_token_ids': None, 'kv_transfer_params': None}


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Length', b'1103'), (b'Content-Type', b'application/json'), (b'Date', b'Fri, 13 Feb 2026 08:50:53 GMT'), (b'Ngrok-Agent-Ips', b'34.13.245.16'), (b'Server', b'uvicorn')])
INFO:httpx:HTTP Request: POST https://nontheatrical-judiciarily-susanne.ngrok-free.dev/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
DEBUG:app.agents.dialogue:conversation_impact: {'npc_stats': {'player': {'affection': 0, 'fear': 0, 'humanity': 0}, 'stepmother': {'affection': 0, 'fear': 0, 'humanity': 0}}, 'event_description': ['플레이어가 새엄마 엘리노어에게 해칠지 묻자, 엘리노어는 가족으로서 아낀다고 말하며 따뜻한 분위기를 조성함.']}
INFO:app.llm.engine:vLLM에 의한 generate 시도
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'

resp : <Response [200 OK]>
data : {'id': 'chatcmpl-9485fceb41e774f1', 'object': 'chat.completion', 'created': 1770972653, 'model': 'kakaocorp/kanana-1.5-8b-instruct-2505', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "npc_stats": {\n    "player": {"affection": 0, "critical_hits": 0, "fear": 0, "humanity": 0, "minus_hits": 0, "plus_hits": 0},\n    "stepmother": {"affection": 0, "critical_hits": 0, "fear": 0, "humanity": 0, "minus_hits": 0, "plus_hits": 0}\n  },\n  "event_description": ["플레이어가 새엄마 엘리노어에게 해칠지 묻자, 엘리노어는 가족으로서 아낀다고 말하며 따뜻한 분위기를 조성함."]\n}\n```', 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [], 'reasoning': None, 'reasoning_content': None}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None, 'token_ids': None}], 'service_tier': None, 'system_fingerprint': None, 'usage': {'prompt_tokens': 573, 'total_tokens': 730, 'completion_tokens': 157, 'prompt_tokens_details': None}, 'prompt_log

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Length', b'620'), (b'Content-Type', b'application/json'), (b'Date', b'Fri, 13 Feb 2026 08:50:55 GMT'), (b'Ngrok-Agent-Ips', b'34.13.245.16'), (b'Server', b'uvicorn')])
INFO:httpx:HTTP Request: POST https://nontheatrical-judiciarily-susanne.ngrok-free.dev/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
DEBUG:app.agents.memory:add_memory: npc=stepmother type=dialogue importance=10.0
INFO:app.rule_engine:[RuleEngine] 규칙 적용: investigate_increase_suspicion (intent=investigate)
DEBUG:app.rule_engine:[RuleEngine] npc_memory_rewrite: stepmother.observation (mode=notice_unusual_behavior) - 미구현, 스킵
DEBUG:app.rule_engine:[RuleEngine] var_add: suspicion_level += 2
DEBUG:app.rule_e

resp : <Response [200 OK]>
data : {'id': 'chatcmpl-970c4e2a30d47631', 'object': 'chat.completion', 'created': 1770972655, 'model': 'kakaocorp/kanana-1.5-8b-instruct-2505', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '10', 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [], 'reasoning': None, 'reasoning_content': None}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None, 'token_ids': None}], 'service_tier': None, 'system_fingerprint': None, 'usage': {'prompt_tokens': 134, 'total_tokens': 136, 'completion_tokens': 2, 'prompt_tokens_details': None}, 'prompt_logprobs': None, 'prompt_token_ids': None, 'kv_transfer_params': None}


ToolResult(state_delta={'vars': {'suspicion_level': 2, 'humanity': 1}, 'npc_stats': {'stepmother': {'affection': 0, 'fear': 0, 'humanity': 0}}, 'triggered_events': [], 'turn_increment': 1}, event_description=['플레이어가 새엄마 엘리노어에게 해칠지 묻자, 엘리노어는 가족으로서 아낀다고 말하며 따뜻한 분위기를 조성함.'], intent='investigate', npc_response='아, 그런! 생각은!! 하지 말아요! 저는 가족의 일원으로서 당신을 아끼고 있답니다?!! 있답니다?!! 있답니다?!! 따뜻한 차 한 잔, 얼마나 좋은가요?! 이야기를 나누어요!!!', npc_id='stepmother', item_id=None)

In [29]:
# ── Step 5: Delta 적용 ──
world_after = _apply_delta(world_state, tool_result.state_delta, assets)
world_after
# turn 1 -> 2
# 대화 memory 추가

WorldStatePipeline(turn=2, npcs={'stepmother': NPCState(npc_id='stepmother', status=<NPCStatus.ALIVE: 'alive'>, stats={'affection': 50, 'fear': 80, 'humanity': 0, 'plus_hits': 0, 'minus_hits': 0, 'critical_hits': 0}, memory={'memory_stream': [{'memory_id': 'a25c4993-a889-4b1d-9f46-fba9dc0d51cb', 'npc_id': 'stepmother', 'description': '플레이어와(과) 대화함. 상대 발언: 저를 해칠거에요....?', 'creation_turn': 1, 'last_access_turn': 1, 'importance_score': 10.0, 'memory_type': 'dialogue', 'metadata': {}}]}), 'stepfather': NPCState(npc_id='stepfather', status=<NPCStatus.ALIVE: 'alive'>, stats={'affection': 30, 'fear': 60, 'humanity': 20}, memory={}), 'brother': NPCState(npc_id='brother', status=<NPCStatus.ALIVE: 'alive'>, stats={'affection': 80, 'fear': 40, 'humanity': 50}, memory={'memory_stream': [{'memory_id': '9b2903bb-98e4-470f-a869-c9ff4b32b10c', 'npc_id': 'brother', 'description': '[비밀 발각] Broken Joint (부러진 관절): 동생의 팔꿈치 안쪽에서 미세한 기계 소음과 함께 찢어진 천 사이로 나무 골격이 보였다는 사실.', 'creation_turn': 1, 'last_access_turn

In [30]:
# ── Step 5.5: ItemAcquirer - 자동 아이템 획득 스캔 ──
from app.item_acquirer import get_item_acquirer
acquirer = get_item_acquirer()
acq_result = acquirer.scan(world_after, assets)
if acq_result.newly_acquired:
    world_after = _apply_delta(world_after, acq_result.acquisition_delta, assets)
    for acq_item_id in acq_result.newly_acquired:
        acq_item_def = assets.get_item_by_id(acq_item_id)
        acq_item_name = acq_item_def.get("name", acq_item_id) if acq_item_def else acq_item_id
        tool_result.event_description.append(f"'{acq_item_name}'을(를) 발견했다!")
acq_result

AcquisitionResult(newly_acquired=[], acquisition_delta={})

In [31]:
# ── Step 5.6: day_action_log 축적 (밤 가족회의 안건용) ──
day_log_entry = {
    "turn": world_after.turn,
    "input": user_text,
    "intent": tool_result.intent,
    "events": tool_result.event_description,
}
world_after.vars.setdefault("day_action_log", []).append(day_log_entry)
day_log_entry

{'turn': 2,
 'input': '저를 해칠거에요....?\ntarget:stepmother',
 'intent': 'investigate',
 'events': ['플레이어가 새엄마 엘리노어에게 해칠지 묻자, 엘리노어는 가족으로서 아낀다고 말하며 따뜻한 분위기를 조성함.']}

In [32]:
# ── Step 6: EndingChecker - 엔딩 체크 ──
ending_result = check_ending(world_after, assets)
ending_info = None
if ending_result.reached:
    ending_info = {
        "ending_id": ending_result.ending.ending_id,
        "name": ending_result.ending.name,
        "epilogue_prompt": ending_result.ending.epilogue_prompt,
    }
    if ending_result.triggered_delta:
        _apply_delta(world_after, ending_result.triggered_delta, assets)
ending_info, ending_result

(None,
 EndingCheckResult(reached=False, ending=None, triggered_delta=StateDelta(npc_stats={}, flags={}, inventory_add=[], inventory_remove=[], locks={}, vars={}, turn_increment=1, memory_updates={}, next_node=None)))

In [33]:
# ── Step 7: NarrativeLayer - 나레이션 생성 ──
narrative_layer = get_narrative_layer()
if ending_info:
    narrative = narrative_layer.render_ending(
        ending_info,
        world_after,
        assets,
    )
else:
    narrative = narrative_layer.render(
        world_after,
        assets,
        event_description=tool_result.event_description,
        state_delta=tool_result.state_delta,
        npc_response=tool_result.npc_response,
    )
print(narrative)

INFO:app.narrative:Rendering narrative (day phase)
DEBUG:app.narrative:Rendered dialogue: 77 chars


플레이어가 새엄마 엘리노어에게 해칠지 묻자, 엘리노어는 가족으로서 아낀다고 말하며 따뜻한 분위기를 조성함.

인간성이 회복되었다. (+1)


In [34]:
# 강제 엔딩 체크
## "조력자의 희생 (The Sibling's Help)"
## condition: "npc.brother.affection >= 90 and npc.brother.humanity >= 70 and vars.day == 5"

world_state.npcs["brother"].stats["affection"] = 95
world_state.npcs["brother"].stats["humanity"] = 80
world_state.vars["day"] = 5

ending_result = check_ending(world_state, assets)
ending_info = None
if ending_result.reached:
    ending_info = {
        "ending_id": ending_result.ending.ending_id,
        "name": ending_result.ending.name,
        "epilogue_prompt": ending_result.ending.epilogue_prompt,
    }
    if ending_result.triggered_delta:
        _apply_delta(world_after, ending_result.triggered_delta, assets)

print(f"ending_info : {ending_info}")
print(f"ending_result : {ending_result}")

NameError: name 'StateDelta' is not defined

In [ ]:
narrative_layer = get_narrative_layer()
if ending_info:
    narrative = narrative_layer.render_ending(
        ending_info,
        world_state,
        assets,
    )
else:
    narrative = narrative_layer.render(
        world_state,
        assets,
        event_description=tool_result.event_description,
        state_delta=tool_result.state_delta,
        npc_response=tool_result.npc_response,
    )
print(narrative)

INFO:app.narrative:Rendering ending: sibling_sacrifice
DEBUG:app.narrative:Rendered ending dialogue: 204 chars



【 조력자의 희생 (The Sibling's Help) 】

...동생의 슬픈 미소와 주인공의 죄책감, 눈물 젖은 탈출을 감성적으로 묘사하라....

이것이 당신이 선택한 결말이다.

[2/50턴 - 인간성 100]



In [ ]:
# ── Step 8: WorldStatePipeline → DB 반영 ──


In [ ]:
# Game 저장

In [ ]:
# 리턴

# 밤 파이프라인 테스트

In [ ]:
# ── Step 4: NightController - 밤 턴 실행 ──
night_controller = get_night_controller()
night_result = night_controller.process(world_state, assets)
night_result

TypeError: UnifiedLLMEngine.generate() takes 1 positional argument but 2 were given